In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, GRU, CuDNNGRU, Bidirectional, Masking, Dense, InputLayer, Dropout, BatchNormalization, Reshape, Flatten, Conv1D, AveragePooling1D, Activation
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split
from util import modified_SMAPE
from losses import SMAPE_loss
from callbacks import EvaluateSMAPE
from tqdm import tqdm, trange, tnrange
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.decomposition import PCA
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
demograpgics = pd.read_csv('datasets/exam-1/demographics.csv')
cc = pd.read_csv('datasets/exam-1/cc.csv')
cc.sort_values(by=['cc_no', 'pos_dt'], inplace=True)
kplus = pd.read_csv('datasets/exam-1/kplus.csv')
kplus.sort_values(by=['id', 'sunday'], inplace=True)
train_set = pd.read_csv('datasets/exam-1/train.csv')
test_set = pd.read_csv('datasets/exam-1/test.csv')

In [3]:
train_scaler = StandardScaler()
scaled_train_set = train_set.copy()
scaled_train_set.set_index('id', inplace=True)
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'], axis=2))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


In [4]:
scaled_train_set.head()

,income
id,
1,-0.525099
2,1.090529
3,-0.356022
4,0.245142
5,-0.562672


# KPLUS

In [5]:
n_transaction_threshold = 8

train_kplus = kplus.copy()
grouped = train_kplus.groupby('id')
trainable_ids = grouped.size().keys()[grouped.size() >= n_transaction_threshold]
train_kplus = train_kplus[train_kplus['id'].isin(trainable_ids)]

In [6]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
100625,2,2018-01-07,2,600
100629,2,2018-01-14,3,3200
100641,2,2018-01-21,2,600
100644,2,2018-01-28,6,3000
100643,2,2018-02-04,4,13700


In [7]:
Scaler = StandardScaler

In [8]:
scaler_x = Scaler()
train_kplus[['kp_txn_count', 'kp_txn_amt']] = scaler_x.fit_transform(train_kplus[['kp_txn_count', 'kp_txn_amt']])
train_kplus = train_kplus[train_kplus['id'] <= 50000]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
train_kplus.head()

,id,sunday,kp_txn_count,kp_txn_amt
100625,2,2018-01-07,-0.404434,-0.218776
100629,2,2018-01-14,-0.237545,-0.198626
100641,2,2018-01-21,-0.404434,-0.218776
100644,2,2018-01-28,0.263124,-0.200176
100643,2,2018-02-04,-0.070655,-0.117253


In [10]:
def create_sequence(group, padding_value):
    origin = sunday_id_hash[group.iloc[0]['sunday']]

    seq = group[['kp_txn_amt', 'kp_txn_count']].to_numpy()
    pre_padding = np.ones((origin ,2), dtype=np.float32) * padding_value
    post_padding = np.ones((max_len - origin - len(seq),2), dtype=np.float32) * padding_value
    return np.concatenate((pre_padding, seq, post_padding), axis=0)

In [11]:
def create_kplus_x(kplus, padding_value,):
    sunday_id_hash = {sunday:i for i, sunday in enumerate(kplus.groupby('sunday', sort=True).groups.keys())}
    max_len = len(sunday_id_hash)
    padding_value = float(-100)

    xs = np.array([create_sequence(group, padding_value) for _, group in kplus.groupby('id')])
    return xs

In [12]:
sunday_id_hash = {sunday:i for i, sunday in enumerate(kplus.groupby('sunday', sort=True).groups.keys())}
max_len = len(sunday_id_hash)
padding_value = float(-100)

id_grouped = kplus.groupby('id')
accept_ids = pd.Series(list(id_grouped.groups.keys()))

xs = create_kplus_x(train_kplus, padding_value)
ys = scaled_train_set.loc[train_kplus['id'].drop_duplicates().to_numpy()].to_numpy()

In [26]:
xs[-1]

array([[-2.17225960e-01, -5.71324028e-01],
       [-2.02501282e-01, -5.71324028e-01],
       [-2.18775926e-01, -5.71324028e-01],
       [-2.15675994e-01, -5.71324028e-01],
       [-2.15675994e-01, -5.71324028e-01],
       [-2.14901011e-01, -5.71324028e-01],
       [-9.55536278e-02, -5.71324028e-01],
       [-1.57303783e-02, -5.71324028e-01],
       [-2.42551914e-02, -4.04434454e-01],
       [-2.18775926e-01, -5.71324028e-01],
       [-2.11026095e-01, -5.71324028e-01],
       [-1.38177693e-01, -4.04434454e-01],
       [-1.39727659e-01, -2.37544879e-01],
       [-2.15675994e-01, -5.71324028e-01],
       [-1.71501962e-01, -4.04434454e-01],
       [ 2.22437889e-02, -7.06553046e-02],
       [-2.00176333e-01, -5.71324028e-01],
       [-1.07953356e-01, -5.71324028e-01],
       [-1.69177013e-01, -5.71324028e-01],
       [-1.00000000e+02, -1.00000000e+02],
       [-1.00000000e+02, -1.00000000e+02],
       [-1.00000000e+02, -1.00000000e+02],
       [-1.00000000e+02, -1.00000000e+02],
       [-1.

In [30]:
ys[-1]

array([-0.65660364])

In [14]:
base_activation = 'tanh'
model = Sequential()
model.add(InputLayer((25,2)))
model.add(Masking(padding_value, ))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(GRU(50, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(GRU(25, activation=base_activation, return_sequences=True))
model.add(Dropout(0.5))
model.add(GRU(25, activation=base_activation))
# model.add(Flatten())
# model.add(Dense(100))
model.add(Dense(1, activation='linear'))

In [15]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [17]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25, 2)             0         
_________________________________________________________________
masking_2 (Masking)          (None, 25, 2)             0         
_________________________________________________________________
gru_6 (GRU)                  (None, 25, 50)            7950      
_________________________________________________________________
gru_7 (GRU)                  (None, 25, 50)            15150     
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 50)            0         
_________________________________________________________________
gru_8 (GRU)                  (None, 25, 25)            5700      
_________________________________________________________________
gru_9 (GRU)                  (None, 25, 25)            3825      
__________

In [18]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [20]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler, use_SMAPE_loss=True),
                     TensorBoard(log_dir='logs/gru')],
          initial_epoch=0,
         epochs=10000)

Instructions for updating:
Use tf.cast instead.
Train on 27725 samples, validate on 6932 samples
Epoch 1/10000
27725/27725 [==============================] - 58s 2ms/step - loss: 0.0896 - val_loss: 0.0827
Epoch 1 | val-SMAPE: 91.72752684189692
Epoch 2/10000
27725/27725 [==============================] - 71s 3ms/step - loss: 0.0849 - val_loss: 0.0796
Epoch 2 | val-SMAPE: 92.04023577019237
Epoch 3/10000
 4256/27725 [===>..........................] - ETA: 52s - loss: 0.0854

KeyboardInterrupt: 

# Credit

In [75]:
cc_id = pd.merge(cc, demograpgics[['id','cc_no']], on='cc_no')

In [76]:
cc_id['count'] = 1

In [77]:
cc_id.head(10)

,cc_no,pos_dt,cc_txn_amt,id,count
0,2,2018-03-10,800,2,1
1,2,2018-03-12,3800,2,1
2,2,2018-04-27,14700,2,1
3,2,2018-04-29,4000,2,1
4,2,2018-05-07,800,2,1
5,2,2018-05-14,800,2,1
6,2,2018-06-04,1000,2,1
7,2,2018-06-11,1000,2,1
8,4,2018-05-11,20000,4,1
9,4,2018-05-11,30000,4,1


In [78]:
grouped = cc_id.groupby(['id', 'pos_dt'])

In [79]:
cc_persons = grouped.sum().reset_index()
cc_persons.head(10)

,id,pos_dt,cc_no,cc_txn_amt,count
0,1,2018-01-20,98397,4700,1
1,1,2018-02-17,196794,20000,2
2,1,2018-05-13,98397,10000,1
3,1,2018-06-14,98397,10000,1
4,2,2018-01-04,9740,1600,1
5,2,2018-01-28,29220,3500,3
6,2,2018-01-29,9740,1500,1
7,2,2018-02-04,9740,1100,1
8,2,2018-02-11,9740,800,1
9,2,2018-02-19,9740,800,1


In [80]:
Scaler = StandardScaler
cc_scaler = Scaler()
train_scaler = Scaler()
cc_persons[['cc_txn_amt', 'count']] = cc_scaler.fit_transform(cc_persons[['cc_txn_amt', 'count']])
scaled_train_set = train_set.copy()
scaled_train_set['income'] = train_scaler.fit_transform(np.expand_dims(scaled_train_set['income'].to_numpy(), axis=1))

In [54]:
cc_persons.head()

,id,pos_dt,cc_no,cc_txn_amt,count
0,1,2018-01-20,98397,0.009926,-0.366991
1,1,2018-02-17,196794,0.421319,1.115008
2,1,2018-05-13,98397,0.152435,-0.366991
3,1,2018-06-14,98397,0.152435,-0.366991
4,2,2018-01-04,9740,-0.073428,-0.366991


In [55]:
scaled_train_set.head()

,id,income
0,1,-0.525099
1,2,1.090529
2,3,-0.356022
3,4,0.245142
4,5,-0.562672


In [66]:
cc_persons = cc_persons[cc_persons['id'] <= 50000]

In [84]:
origin_timestamp = pd.Timestamp(cc['pos_dt'].min())
max_len_seq = (pd.Timestamp(cc['pos_dt'].max()) - origin_timestamp).days + 1
cc_persons['pos_dt_index'] = cc_persons.apply(lambda row: (pd.Timestamp(row['pos_dt']) - origin_timestamp).days, axis=1)

In [68]:
# padding_value = float(-100)
padding_value = scaler_x.transform([[0,0]])
xs = [] # [[cc_txn_amt, count], ...]
ys = []
train_id_set = scaled_train_set.set_index('id')
empty_seq = np.ones((max_len_seq, 2), dtype=np.float32) * padding_value
for _id, group in cc_persons.groupby('id'):
    seq = empty_seq.copy()
    seq[group['pos_dt_index'].to_numpy()] = group[['cc_txn_amt', 'count']].to_numpy()
    
    xs.append(seq)
    ys.append(train_id_set.loc[_id]['income'])
xs = np.asarray(xs)
ys = np.asarray(ys)

In [ ]:
xs

In [ ]:
ys

In [69]:
base_activation = 'tanh'
RNN = GRU
model = Sequential()
model.add(InputLayer((max_len_seq,2)))
model.add(Bidirectional(RNN(128, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(RNN(128, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(RNN(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Bidirectional(RNN(64, return_sequences=True)))
model.add(Activation(base_activation))
model.add(Dropout(0.5))
model.add(Bidirectional(RNN(32, )))
model.add(Activation(base_activation))
model.add(Dense(1, activation='linear'))

In [45]:
model = Sequential()
base_activation = 'relu'
model.add(Conv1D(32, kernel_size=3, activation=base_activation, input_shape=(max_len_seq,2)))
model.add(Conv1D(32, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Conv1D(16, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Conv1D(8, kernel_size=3, activation=base_activation))
model.add(Dropout(0.5))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
model.add(Conv1D(4, kernel_size=3, activation=base_activation))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, activation='linear'))

In [63]:
model = Sequential()
model.add(InputLayer((max_len_seq,2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [70]:
# opt = SGD(lr=4e-2, momentum=0.1)
opt = Adam()

In [71]:
model.compile(loss=SMAPE_loss(train_scaler), optimizer=opt )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 181, 2)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 181, 256)          100608    
_________________________________________________________________
activation_1 (Activation)    (None, 181, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 181, 256)          295680    
_________________________________________________________________
activation_2 (Activation)    (None, 181, 256)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 181, 256)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 181, 128)          123264    
__________

In [72]:
x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2)

In [73]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          shuffle=True,
          batch_size=32,
          callbacks=[EvaluateSMAPE(x_val, y_val, 'val', scaler_y=train_scaler),
                     TensorBoard(log_dir='logs/cc_gru_smape')],
          initial_epoch=0,
         epochs=10000)

Instructions for updating:
Use tf.cast instead.
Train on 31960 samples, validate on 7990 samples
Epoch 1/10000
31960/31960 [==============================] - 797s 25ms/step - loss: 0.1061 - val_loss: 0.1036
Epoch 1 | val-SMAPE: 89.63577463213696
Epoch 2/10000
  672/31960 [..............................] - ETA: 13:11 - loss: 0.1024

KeyboardInterrupt: 

In [74]:
model.save('logs/cc_gru_smape/smape:89.635.h5')

# Predict

In [85]:
cc_test_persons = pd.merge(cc_persons, test_set, on='id', how='right').sort_values('id')

In [90]:
cc_test_persons.head()

,id,pos_dt,cc_no,cc_txn_amt,count,pos_dt_index
0,50001,2018-02-11,68552.0,0.448207,-0.366991,41.0
1,50001,2018-02-27,68552.0,-0.103005,-0.366991,57.0
2,50001,2018-04-02,68552.0,0.039503,-0.366991,91.0
3,50001,2018-04-28,68552.0,-0.097628,-0.366991,117.0
4,50001,2018-04-29,68552.0,-0.057295,-0.366991,118.0


In [115]:
xs = []
for _id, group in cc_test_persons.groupby('id'):
    seq = empty_seq.copy()
    if not group.isna().values.any():
        seq[group['pos_dt_index'].to_numpy().astype(np.int)] = group[['cc_txn_amt', 'count']].to_numpy()
    xs.append(seq)
xs = np.array(xs)

In [117]:
y_pred = model.predict(xs, verbose=1)

15000/15000 [==============================] - 68s 5ms/step


In [118]:
output = test_set.copy()
output['income'] = train_scaler.inverse_transform(y_pred)

In [120]:
output.head(10)

,id,income
0,50001,27878.587891
1,50002,29017.464844
2,50003,27876.582031
3,50004,27906.773438
4,50005,29286.541016
5,50006,30806.785156
6,50007,27967.406250
7,50008,32748.785156
8,50009,28767.205078
9,50010,27876.582031


In [121]:
output.to_csv('cc_gru_smape.csv')